# Avaliação modelo DTLFE pré-treinado

---


# Configurações

In [1]:
import os
import sys
import gc
import json
from pprint import pprint
from collections import Counter
import copy
import warnings
warnings.filterwarnings(action="ignore")

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from IPython.display import display

from tqdm import *

from pretty_confusion_matrix import *

# TODO: implementar rotina na classe PyNILM.utils
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

def listar_variaveis_memoria(ambiente):
    print("* Variáveis instanciadas em memória:")
    print("---")
    total = 0
    for name, size in sorted(((name, sys.getsizeof(value)) for name, value in ambiente.items()),
                             key= lambda x: -x[1])[:10]:
        total += size
        print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))
    print("---")
    print("Total:", sizeof_fmt(total))
    
# TODO: implementar na classe utils
def highlight_col(x):
    r = 'background-color: #D9D9D9'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, -2] = r
    return df1   

In [2]:
# # CONSTANTES FUNDAMENTAIS DE ORGANIZACAO DE PASTAS/ARQUIVOS
RESIDENCIA = 3

MODELO_PRETREINADO = "VGG16"

# Path do arquivo H5 (base REDD ja preparada p/ NILMTK) e outros insumos fundamentais
caminho_dados = "D:/Projetos/phd-thesis/datasets/"

# Definir diretorios onde iremos salvar os insumos gerados do notebook (dados, imagens, etc.)
caminho_dados_notebook = os.path.join(caminho_dados, "28", MODELO_PRETREINADO) # Num. notebook
if not os.path.isdir(caminho_dados_notebook):
    os.makedirs(caminho_dados_notebook)
caminho_imagens_notebook = os.path.join(caminho_dados_notebook, "imagens") # Num. notebook
if not os.path.isdir(caminho_imagens_notebook):
    os.makedirs(caminho_imagens_notebook)


In [3]:
from datetime import datetime, timedelta
from dateutil.parser import parse as date_parser

from matplotlib import rcParams
import matplotlib.pyplot as plt
from six import iteritems

from nilmtk import DataSet, TimeFrame, MeterGroup, HDFDataStore
from nilmtk.legacy.disaggregate import CombinatorialOptimisation, FHMM
import nilmtk.utils

%matplotlib inline

# Dados

## Base REDD

In [4]:
# Path do arquivo H5 (base REDD ja preparada p/ NILMTK)
caminho_redd = os.path.join(caminho_dados, "REDD/low_freq")
caminho_ukdale = os.path.join(caminho_dados, "UK-DALE")

# Path completo do arquivo REDD/UKDALE
arquivo_dataset_redd = os.path.join(caminho_redd, "redd.h5")
arquivo_dataset_ukdale = os.path.join(caminho_ukdale, "ukdale.h5")

# VARIAVEL AUXILIAR
# Path dos arquivos relacionados as janelas
caminho_janelas = os.path.join(caminho_redd, "../../phd")
if not os.path.isdir(caminho_janelas):
    os.makedirs(caminho_janelas)

# Gerar arquivo H5 (Nilmtk) do dataset REDD, caso n exista
if not os.path.isfile(arquivo_dataset_redd):
    from nilmtk.dataset_converters import convert_redd
    
    print("Gerando arquivo H5 (NILMTK) da base REDD, aguarde...")
    print("-----")
    convert_redd(caminho_redd, arquivo_dataset_redd)

# Carregando dataset REDD no objeto NILMTK
# Exemplo de carregamento da base REDD no NILMTK
import h5py # * Evitar erro de incompatibilidade entre h5py e nilmtk
from nilmtk import DataSet
from nilmtk.utils import print_dict

redd = DataSet(arquivo_dataset_redd)
print("NILMTK -> Detalhes sobre o dataset REDD:")
print_dict(redd.metadata)
print()

NILMTK -> Detalhes sobre o dataset REDD:


## Base UK-DALE

In [5]:
ukdale = DataSet(arquivo_dataset_ukdale)
print("NILMTK -> Detalhes sobre o dataset UK-DALE:")
print_dict(ukdale.metadata)
print()

NILMTK -> Detalhes sobre o dataset UK-DALE:


**IMPORTANTE:** O fine-tuning do modelo foi realizado utilizando como dados:

1. Base REDD: `30 dias iniciais` de medições de cada aparelho de interesseo;
2. Base UK-DALE: `60 dias iniciais` de registros de cada aparelho de interesse. 

## Melhores Combinações de Taxas e Janelas para cada Aparelho (artigo IEEE-2021)

In [6]:
df_melhores_taxas_janelas = pd.read_csv(
    os.path.join(caminho_dados, "27", "melhores_taxa_janela_aparelhos_ieee2021.csv"), 
    index_col=0)[['carga', 'taxa_amostragem', 'janela']]
df_melhores_taxas_janelas

,carga,taxa_amostragem,janela
0,dish_washer - 9,2,2040
1,fridge - 7,2,720
2,microwave - 16,2,900
3,washer_dryer - 13,2,90
4,washer_dryer - 14,3,2040


In [7]:
# Melhores configurações de janelas por aparelhos, conforme 
#  artigo IEEE (https://ieeexplore.ieee.org/mediastore_new/IEEE/content/media/6287639/9312710/9564044/ferna.t3-3118947-large.gif),
#  independente do nomenclatura/instancia da base REDD.
with open(
    os.path.join(caminho_dados, "27", "config_aparelhos_janelas_ieee2021.json"), 
    'r') as f:
    aparelhos_janelas = json.load(f)
aparelhos_janelas
# * obs.: como a base redd (artigo) possui dois washer_dryer's, onde verificou-se
#         duas configurações distintas de janelas (90 e 2040), vamos usar a maior.

{'dish': 2040,
 'fridge': 720,
 'microwave': 900,
 'washer_dryer': 2040,
 'washing_machine': 2040}

In [8]:
# TODO: 
# - Desenvolver módulo da metodologia na lib PyNILM 

## Parâmetros de RP dos Aparelhos (estudo 18)

In [9]:
# Carregando arquivos de parametros, caso n estejam (kernel reiniciado)
if not 'parametros_rp_aparelho' in locals():
    with open(os.path.join(caminho_dados, "18", "parametros_rp_aparelho.json"),'r') as arquivo:
        parametros_rp_aparelho = json.load(arquivo)

## Ambiente e Funções Auxiliares

In [10]:
# from PyNILM.dados.janelas import Janelas
from PyNILM.dados.janelas import Janelas
from PyNILM.dados.utils import *

from PyNILM.avaliacao.metricas import *
from PyNILM.avaliacao.graficos import *
from PyNILM.avaliacao.analises import *

from PyNILM.modelos.utils import *
from PyNILM.modelos.dlafe import DLAFE
from PyNILM.modelos.rqa import RQA

# Inicializar uso GPU
start_tf_session(memory_limit=int(1024*4))

Virtual devices cannot be modified after being initialized


## Configurações do Experimento

In [11]:
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost.sklearn import XGBClassifier

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=SEED)

# aparelhos = [
#     'dish_washer - 9',
#     'fridge - 7',
#     'microwave - 16',
#     'washer_dryer - 13', 
#     'washer_dryer - 14'
# ]

TAXA = 2 # Fixa

# IMPORTANTE: Agora o modelo extrator será o pre-treinado
# modelo_extrator = transfer_learning.vgg16.VGG16(
#             weights='imagenet', 
#             include_top=False,
#             pooling='avg'
#         )
preprocessamento_extrator = transfer_learning.vgg16.preprocess_input

## Carregando os dados

In [12]:
# https://notebook.community/jaduimstra/nilmtk/docs/manual/user_guide/elecmeter_and_metergroup

In [13]:
# atraso_inicial = 30
# periodo = 30

# residencia = redd.buildings[RESIDENCIA]

# # Gerar janelas para cada canal/aparelho
# print("* Gerando janelas para cada canal/aparelho...")

# # inicio_intervalo =  datetime.strptime(inicio_intervalo, '%Y-%m-%d %H:%M:%S').date()
# # fim_intervalo = datetime.strptime(fim_intervalo, '%Y-%m-%d %H:%M:%S').date()
# print('Residencia:', residencia)

# for e in residencia.elec.all_meters():

#     # Selecionando canal/aparelho
#     # e = self.residencia.elec[e_i]

#     # Normalizar nome aparelho/canal de medicao
#     aparelho = e.label().lower().replace(" ", "_")

#     # if not any(map(aparelho.__contains__, aparelhos.keys())) or 'site_meter' in aparelho:
#     #     continue

#     tamanho_janela = 1080

#     # # Extraindo medicoes de energia da carga
#     # power = np.array(e.power_series_all_data(sample_period=taxa_amostral).values[:limite_serie])
    
#     # Extraindo medicoes de energia da carga (toda a serie)
#     consumo_aparelho = e.power_series_all_data(sample_period=TAXA).replace(np.nan, 0) # Remover nan (por zero)

#     # Definindo periodo/janela de analise (consumo individual e agregado)
#     if aparelho not in 'site_meter':
#         inicio_periodo = consumo_aparelho.index[0] + timedelta(days=atraso_inicial)
#         fim_periodo = inicio_periodo + timedelta(days=atraso_inicial+periodo)
        
#         # Selecionando periodo no consumo do aparelho
#         indices_aparelho = consumo_aparelho.index.to_pydatetime()
#         consumo_aparelho = consumo_aparelho[(indices_aparelho >= inicio_periodo) & (indices_aparelho <= fim_periodo)]
#         print(f'Consumo aparelho {aparelho}: de', inicio_periodo, 'a', consumo_aparelho.index[-1],  '|', consumo_aparelho.shape)
    
#         break

In [14]:
def carregar_dados(
    base, taxa_amostral=2, periodo=30, 
    residencias=None, 
    janelas_otimizadas=None,
    janela_padrao=900, 
    top_k_consumo=None,
    ignorar_janelas_vazias=True,
    estatisticas=False, metadados=False, 
    debug=False):
    """
    Função para carregar e preparar as janelas de consumo e ativações dos aparelhos em diferentes residenciais de um dataset.

    Retorno:
        dados (list): lista com as informações (janelas e metadados) dos aparelhos de interesse.
    """
    dados = []

    for r in base.buildings:
        
        if residencias and r not in residencias:
            continue

        if debug: print(f"> Processando residência #{r}...")
        
        # dados[b] = {
        #     'medidores': [],
        #     'aparelhos': []
        # }

        # Consumo agregado da residencia (serie historica)
        # if debug: print("* Extraindo dados de consumo agregado da residencia...")
        consumo_agregado = carregar_dados_consumo_agregado(
            base, taxa_amostral=taxa_amostral, residencia=r, debug=debug)
        indices_agregado = consumo_agregado.index.to_pydatetime()
        if debug: print(f'  - Consumo agregado: de', consumo_agregado.index[0], 'a', consumo_agregado.index[-1])

        residencia = base.buildings[r]

        if top_k_consumo:
            if debug: print("* Obtendo aparelhos com maiores consumo de energia...")
            aparelhos_maiores_consumo = {}
            top = residencia.elec.submeters().select_top_k(k=top_k_consumo)
            for m in top.all_meters():
                # Obter janela otimizada (baseado no conhecimento)
                aparelho = m.label().lower().replace(" ", "_") 
                id_aparelho = f"{aparelho} - {m.instance()}"
                tamanho_janela = janela_padrao
                if janelas_otimizadas:
                    for k, v in janelas_otimizadas.items():
                        if k in aparelho:
                            tamanho_janela = v
                            break
                aparelhos_maiores_consumo[id_aparelho] = tamanho_janela

            # if aparelhos:
            #     config_aparelhos = {**aparelhos, **aparelhos_maiores_consumo}
            # else:
            config_aparelhos = aparelhos_maiores_consumo
        else:
            config_aparelhos = janelas_otimizadas

        print('config_aparelhos', config_aparelhos)

        # Gerar janelas para cada canal/aparelho
        if debug: print("\n* Gerando janelas de consumo para cada canal/aparelho...")

        # inicio_intervalo =  datetime.strptime(inicio_intervalo, '%Y-%m-%d %H:%M:%S').date()
        # fim_intervalo = datetime.strptime(fim_intervalo, '%Y-%m-%d %H:%M:%S').date()
        #if debug: print('Residencia:', residencia)
        
        for e in residencia.elec.all_meters():

            # Selecionando canal/aparelho
            # e = self.residencia.elec[e_i]

            # Normalizar nome aparelho/canal de medicao
            aparelho = e.label().lower().replace(" ", "_")
            id_aparelho = f"{aparelho} - {e.instance()}"

            # Validar se aparelho é liberado para exportacao (modo fine-tuning ou avaliacao)
            if top_k_consumo:

                #if not any(map(aparelho.__contains__, config_aparelhos.keys())) \
                if id_aparelho not in config_aparelhos.keys() \
                    or 'site_meter' in aparelho:
                    continue
            else:
                if not any(map(aparelho.__contains__, config_aparelhos.keys())) or 'site_meter' in aparelho:
                    continue

            tamanho_janela = janela_padrao
            for k, v in config_aparelhos.items():
                if k.split(' - ')[0] in aparelho:
                    tamanho_janela = v
                    break

            try:

                # # Extraindo medicoes de energia da carga
                # power = np.array(e.power_series_all_data(sample_period=taxa_amostral).values[:limite_serie])
                
                # Extraindo medicoes de energia da carga (toda a serie)
                consumo_aparelho = e.power_series_all_data(sample_period=taxa_amostral).replace(np.nan, 0) # Remover nan (por zero)

                # Definindo periodo/janela de analise (consumo individual e agregado)
                if aparelho not in 'site_meter':
                    inicio_periodo = consumo_aparelho.index[0]
                    fim_periodo = consumo_aparelho.index[0] + timedelta(days=periodo)
                    
                    # Selecionando periodo no consumo do aparelho
                    indices_aparelho = consumo_aparelho.index.to_pydatetime()
                    consumo_aparelho = consumo_aparelho[(indices_aparelho >= inicio_periodo) & (indices_aparelho <= fim_periodo)]
                    if debug: print(f'   - Consumo aparelho {aparelho}: de', consumo_aparelho.index[0], 'a', consumo_aparelho.index[-1], 
                                '|', consumo_aparelho.shape, 
                                '>>>', consumo_agregado.values.min(), consumo_agregado.values.max(),
                                    consumo_agregado.values.mean(), consumo_agregado.values.std())

                    # Selecionando periodo no consumo agregado (mesmo range de analise para as janelas)
                    consumo_agregado_aparelho = consumo_agregado[(indices_agregado >= inicio_periodo) & (indices_agregado <= fim_periodo)]
                    if debug: 
                        print(f'  - Consumo agregado/aparelho: de ', consumo_agregado_aparelho.index[0], 'a', consumo_agregado_aparelho.index[-1],
                                '|', consumo_agregado_aparelho.shape, 
                                '>>>', consumo_agregado_aparelho.values.min(), consumo_agregado_aparelho.values.max(),
                                    consumo_agregado_aparelho.values.mean(), consumo_agregado_aparelho.values.std())
                        print(f'  - Shapes sincronizados?', consumo_agregado_aparelho.shape == consumo_aparelho.shape)

                    # print(' -', aparelho, '=', inicio_periodo, 'a', fim_periodo, '|', tamanho_janela )

                    # TODO: dividir em janelas com indices -> np.array_split(dados, dados.shape[0] // (720 - 1))
                    
                    # Calculando tamanho máximo da série (padding, dependendo tamanho janeka)
                    limite_serie = int(len(consumo_aparelho.values) / tamanho_janela) * tamanho_janela

                    # Garantindo limite da serie valido (caber dentro do reshape do tamanho janela)
                    while limite_serie % tamanho_janela != 0:
                        limite_serie -= 1
                    
                    # Encaixando medicao dentro do tamanho de janelas (p/ fazer reshape)
                    # power = power.values[:limite_serie]
                    consumo_aparelho = consumo_aparelho.iloc[:limite_serie]
                    consumo_agregado_aparelho = consumo_agregado_aparelho.iloc[:limite_serie]

                    # Gerando máscara de status (ativo ou não), considerando ruido da carga
                    # ou rede na medição (threshod)
                    # status = power > e.on_power_threshold()
                    ativacoes = (consumo_aparelho >= e.on_power_threshold()).astype(int)

                    # # Dividindo em janelas (consumo energetico individual e agregado, bem como ativacoes)
                    # windows_series = power.reshape(-1, tamanho_janela)
                    # windows_status = status.reshape(-1, tamanho_janela)
                    # janelas_aparelho = np.array_split(consumo_aparelho, consumo_aparelho.shape[0] // (tamanho_janela - 1))
                    janelas_consumo_aparelho = np.vstack(
                        np.array_split(
                            consumo_aparelho, 
                            consumo_aparelho.shape[0] // tamanho_janela
                        )
                    )
                    janelas_agregado_aparelho = np.vstack(
                        np.array_split(
                            consumo_agregado_aparelho, 
                            consumo_agregado_aparelho.shape[0] // tamanho_janela
                        )
                    )
                    janelas_ativacoes = np.vstack(
                        np.array_split(
                            ativacoes, 
                            ativacoes.shape[0] // tamanho_janela
                            )
                    )
                    
                    if ignorar_janelas_vazias:
                        janelas_validas = [True if np.sum(j) > 0 else False for j in janelas_agregado_aparelho]

                        janelas_agregado_aparelho = janelas_agregado_aparelho[janelas_validas]
                        janelas_consumo_aparelho = janelas_consumo_aparelho[janelas_validas]
                        janelas_ativacoes = janelas_ativacoes[janelas_validas]

                    # # # Remover nan (por zero)
                    # # windows_series = np.nan_to_num(windows_series)

                    # # Extraindo ativacoes
                    # windows_status = np.where(
                    #     np.sum(windows_status, axis=1) > 0, 1, 0
                    # )  # Estado de cada janela, baseado na pré-avaliação da serie
                    ativacao_por_janela = []
                    for w in janelas_ativacoes:
                        ativacao_por_janela.append(1 if w.sum() > 0 else 0)
                    

                    # #     # Calcular rotulos a partir das janelas
                    # #     # Podendo ser:
                    # #     #   - `estado` (denotando carga ATIVA [1] ou INATIVA [0]);
                    # #     #   - `total`(soma da janela);
                    # #     #   - `media`;
                    # #     rotulos = {
                    # #         "total": np.sum(windows_series, axis=1),
                    # #         "media": np.mean(windows_series, axis=1),
                    # #         "estado": np.where(
                    # #             np.sum(windows_status, axis=1) > 0, 1, 0
                    # #         )  # Estado de cada janela, baseado na pré-avaliação da serie
                    # #         # completa, considerando ruido
                    # #     }

                    d = {
                        "aparelho": aparelho,
                        "instancia": e.instance(),
                        "residencia": r,
                        "janelas": janelas_agregado_aparelho,
                        "consumo": janelas_consumo_aparelho,
                        "status": np.array(ativacao_por_janela)
                        }

                    if estatisticas:
                        d["estatisticas"] = {
                            "status": dict(Counter(ativacao_por_janela)),
                            # TODO: validar integridade das janelas (mesmo tamanho) e conversao das janelas para np.array (np.vstack)
                            "consumo_por_janela": {
                                "min": janelas_agregado_aparelho.sum(axis=1).min(),
                                "max": janelas_agregado_aparelho.sum(axis=1).max(),
                                "mean": janelas_agregado_aparelho.sum(axis=1).mean(),
                                "std": janelas_agregado_aparelho.sum(axis=1).std()
                            },
                            "consumo_historico_aparelho": {
                                "min": consumo_aparelho.min(),
                                "max": consumo_aparelho.max(),
                                "mean": consumo_aparelho.mean(),
                                "std": consumo_aparelho.std()
                            }
                        }
                    
                    if metadados:
                        d["metadata"] = e.metadata

                    dados.append(d)

                    # TODO: Sincronizar medidor e aparelho (indices)

                    #     if self.debug: print(f"{aparelho} -> {windows_series.shape}")

            except Exception as ex:
                if debug: print(f"{aparelho}-{e.instance()}: erro ao extrair dados -> {str(ex)}")
                # return {
                #         "aparelho": aparelho,
                #         "instancia": e.instance(),
                #         "residencia": r,
                #         "janelas": janelas_agregado_aparelho,
                #         "consumo": consumo_aparelho,
                #         "status": np.array(ativacao_por_janela)
                #         }
                
    return dados

def carregar_dados_consumo_agregado(base, taxa_amostral=2, residencia=1, debug=False):
    # Consumo agregado = potencia aparente
    # COnsumo individual = potencia ativa
    dados = []

    for b in base.buildings:
            
        if b != residencia:
            continue

        residencia = base.buildings[b]

        # Gerar janelas para cada canal/aparelho
        if debug: print("* Extraindo consumo agregado dos medidores...")

        # inicio_intervalo =  datetime.strptime(inicio_intervalo, '%Y-%m-%d %H:%M:%S').date()
        # fim_intervalo = datetime.strptime(fim_intervalo, '%Y-%m-%d %H:%M:%S').date()
        if debug: print('Residencia:', residencia)
        
        for e in residencia.elec.all_meters():

            # Selecionando canal/aparelho
            # e = self.residencia.elec[e_i]

            # Normalizar nome aparelho/canal de medicao
            aparelho = e.label().lower().replace(" ", "_")

            if aparelho != 'site_meter':
                continue

            try:

                # # Extraindo medicoes de energia da carga
                # power = np.array(e.power_series_all_data(sample_period=taxa_amostral).values[:limite_serie])
                
                # Extraindo medicoes de energia da carga (toda a serie)
                consumo = e.power_series_all_data(sample_period=taxa_amostral)

                # Remover nan (por zero)
                consumo = consumo.replace(np.nan, 0)

                dados.append({
                    "rotulo": aparelho,
                    "instancia": e.instance(),
                    "consumo": consumo
                })

            except Exception as e:
                if debug: print(f"{aparelho}: erro ao extrair dados -> {str(e)}")

    consumo_agregado = pd.concat([c['consumo'] for c in dados], axis=1).sum(1, min_count=1).fillna(0)

    return consumo_agregado

def carregar_dados_OLD(
    base, taxa_amostral=2, 
    atraso_inicial=0, periodo=30, 
    residencias=None, aparelhos=None, 
    estatisticas=False, metadados=False, 
    debug=False):
    """
    Função para carregar e preparar as janelas de consumo e ativações dos aparelhos em diferentes residenciais de um dataset.

    Retorno:
        dados (list): lista com as informações (janelas e metadados) dos aparelhos de interesse.
    """
    dados = []

    for r in base.buildings:
        
        if residencias and r not in residencias:
            continue

        if debug: print(f"> Processando residência #{r}...")
        
        # dados[b] = {
        #     'medidores': [],
        #     'aparelhos': []

        # }

        # Consumo agregado da residencia (serie historica)
        # if debug: print("* Extraindo dados de consumo agregado da residencia...")
        consumo_agregado = carregar_dados_consumo_agregado(
            base, taxa_amostral=taxa_amostral, residencia=r, debug=debug)
        indices_agregado = consumo_agregado.index.to_pydatetime()
        if debug: print(f'  - Consumo agregado: de', consumo_agregado.index[0], 'a', consumo_agregado.index[-1])

        residencia = base.buildings[r]

        # Gerar janelas para cada canal/aparelho
        if debug: print("* Gerando janelas para cada canal/aparelho...")

        # inicio_intervalo =  datetime.strptime(inicio_intervalo, '%Y-%m-%d %H:%M:%S').date()
        # fim_intervalo = datetime.strptime(fim_intervalo, '%Y-%m-%d %H:%M:%S').date()
        if debug: print('Residencia:', residencia)
        
        for e in residencia.elec.all_meters():

            # Selecionando canal/aparelho
            # e = self.residencia.elec[e_i]

            # Normalizar nome aparelho/canal de medicao
            aparelho = e.label().lower().replace(" ", "_")

            if not any(map(aparelho.__contains__, aparelhos.keys())) or 'site_meter' in aparelho:
                continue

            tamanho_janela = 1080
            if aparelhos:
                for k, v in aparelhos.items():
                    if k in aparelho:
                        tamanho_janela = v
                        break

            try:

                # # Extraindo medicoes de energia da carga
                # power = np.array(e.power_series_all_data(sample_period=taxa_amostral).values[:limite_serie])
                
                # Extraindo medicoes de energia da carga (toda a serie)
                consumo_aparelho = e.power_series_all_data(sample_period=taxa_amostral).replace(np.nan, 0) # Remover nan (por zero)

                # Definindo periodo/janela de analise (consumo individual e agregado)
                if aparelho not in 'site_meter':
                    inicio_periodo = consumo_aparelho.index[0] + timedelta(days=atraso_inicial)
                    fim_periodo = inicio_periodo + timedelta(days=atraso_inicial+periodo)
                    
                    # Selecionando periodo no consumo do aparelho
                    indices_aparelho = consumo_aparelho.index.to_pydatetime()
                    consumo_aparelho = consumo_aparelho[(indices_aparelho >= inicio_periodo) & (indices_aparelho <= fim_periodo)]
                    if debug: print(f'Consumo aparelho {aparelho}: de', inicio_periodo, 'a', consumo_aparelho.index[-1], 
                                '|', consumo_aparelho.shape, 
                                '>>>', consumo_agregado.values.min(), consumo_agregado.values.max(),
                                    consumo_agregado.values.mean(), consumo_agregado.values.std())

                    # Selecionando periodo no consumo agregado (mesmo range de analise para as janelas)
                    consumo_agregado_aparelho = consumo_agregado[(indices_agregado >= inicio_periodo) & (indices_agregado <= fim_periodo)]
                    if debug: 
                        print(f'  - Consumo agregado/aparelho: de ', consumo_agregado_aparelho.index[0], 'a', consumo_agregado_aparelho.index[-1],
                                '|', consumo_agregado_aparelho.shape, 
                                '>>>', consumo_agregado_aparelho.values.min(), consumo_agregado_aparelho.values.max(),
                                    consumo_agregado_aparelho.values.mean(), consumo_agregado_aparelho.values.std())
                        print(f'  - Shapes sincronizados?', consumo_agregado_aparelho.shape == consumo_aparelho.shape)

                    # print(' -', aparelho, '=', inicio_periodo, 'a', fim_periodo, '|', tamanho_janela )

                    # TODO: dividir em janelas com indices -> np.array_split(dados, dados.shape[0] // (720 - 1))
                    
                    # Calculando tamanho máximo da série (padding, dependendo tamanho janeka)
                    limite_serie = int(len(consumo_aparelho.values) / tamanho_janela) * tamanho_janela

                    # Garantindo limite da serie valido (caber dentro do reshape do tamanho janela)
                    while limite_serie % tamanho_janela != 0:
                        limite_serie -= 1
                    
                    # Encaixando medicao dentro do tamanho de janelas (p/ fazer reshape)
                    # power = power.values[:limite_serie]
                    consumo_aparelho = consumo_aparelho.iloc[:limite_serie]
                    consumo_agregado_aparelho = consumo_agregado_aparelho.iloc[:limite_serie]

                    # Gerando máscara de status (ativo ou não), considerando ruido da carga
                    # ou rede na medição (threshod)
                    # status = power > e.on_power_threshold()
                    ativacoes = (consumo_aparelho >= e.on_power_threshold()).astype(int)

                    # # Dividindo em janelas (consumo energetico individual e agregado, bem como ativacoes)
                    # windows_series = power.reshape(-1, tamanho_janela)
                    # windows_status = status.reshape(-1, tamanho_janela)
                    # janelas_aparelho = np.array_split(consumo_aparelho, consumo_aparelho.shape[0] // (tamanho_janela - 1))
                    janelas_consumo_aparelho = np.vstack(
                        np.array_split(
                            consumo_aparelho, 
                            consumo_aparelho.shape[0] // tamanho_janela
                        )
                    )
                    janelas_agregado_aparelho = np.vstack(
                        np.array_split(
                            consumo_agregado_aparelho, 
                            consumo_agregado_aparelho.shape[0] // tamanho_janela
                        )
                    )
                    janelas_ativacoes = np.vstack(
                        np.array_split(
                            ativacoes, 
                            ativacoes.shape[0] // tamanho_janela
                            )
                    )

                    # # # Remover nan (por zero)
                    # # windows_series = np.nan_to_num(windows_series)

                    # # Extraindo ativacoes
                    # windows_status = np.where(
                    #     np.sum(windows_status, axis=1) > 0, 1, 0
                    # )  # Estado de cada janela, baseado na pré-avaliação da serie
                    ativacao_por_janela = []
                    for w in janelas_ativacoes:
                        ativacao_por_janela.append(1 if w.sum() > 0 else 0)
                    

                    # #     # Calcular rotulos a partir das janelas
                    # #     # Podendo ser:
                    # #     #   - `estado` (denotando carga ATIVA [1] ou INATIVA [0]);
                    # #     #   - `total`(soma da janela);
                    # #     #   - `media`;
                    # #     rotulos = {
                    # #         "total": np.sum(windows_series, axis=1),
                    # #         "media": np.mean(windows_series, axis=1),
                    # #         "estado": np.where(
                    # #             np.sum(windows_status, axis=1) > 0, 1, 0
                    # #         )  # Estado de cada janela, baseado na pré-avaliação da serie
                    # #         # completa, considerando ruido
                    # #     }

                    d = {
                        "aparelho": aparelho,
                        "instancia": e.instance(),
                        "residencia": r,
                        "janelas": janelas_agregado_aparelho,
                        "consumo": janelas_consumo_aparelho,
                        "status": np.array(ativacao_por_janela)
                        }

                    if estatisticas:
                        d["estatisticas"] = {
                            "status": dict(Counter(ativacao_por_janela)),
                            # TODO: validar integridade das janelas (mesmo tamanho) e conversao das janelas para np.array (np.vstack)
                            "consumo_por_janela": {
                                "min": janelas_agregado_aparelho.sum(axis=1).min(),
                                "max": janelas_agregado_aparelho.sum(axis=1).max(),
                                "mean": janelas_agregado_aparelho.sum(axis=1).mean(),
                                "std": janelas_agregado_aparelho.sum(axis=1).std()
                            },
                            "consumo_historico_aparelho": {
                                "min": consumo_aparelho.min(),
                                "max": consumo_aparelho.max(),
                                "mean": consumo_aparelho.mean(),
                                "std": consumo_aparelho.std()
                            }
                        }
                    
                    if metadados:
                        d["metadata"] = e.metadata

                    dados.append(d)

                    # TODO: Sincronizar medidor e aparelho (indices)

                    #     if self.debug: print(f"{aparelho} -> {windows_series.shape}")

            except Exception as ex:
                if debug: print(f"{aparelho}-{e.instance()}: erro ao extrair dados -> {str(ex)}")
                # return {
                #         "aparelho": aparelho,
                #         "instancia": e.instance(),
                #         "residencia": r,
                #         "janelas": janelas_agregado_aparelho,
                #         "consumo": consumo_aparelho,
                #         "status": np.array(ativacao_por_janela)
                #         }
                
    return dados

def carregar_dados_consumo_agregado(base, taxa_amostral=2, residencia=1, debug=False):
    # Consumo agregado = potencia aparente
    # COnsumo individual = potencia ativa
    dados = []

    for b in base.buildings:
            
        if b != residencia:
            continue

        residencia = base.buildings[b]

        # Gerar janelas para cada canal/aparelho
        if debug: print("* Extraindo consumo agregado dos medidores...")

        # inicio_intervalo =  datetime.strptime(inicio_intervalo, '%Y-%m-%d %H:%M:%S').date()
        # fim_intervalo = datetime.strptime(fim_intervalo, '%Y-%m-%d %H:%M:%S').date()
        if debug: print('Residencia:', residencia)
        
        for e in residencia.elec.all_meters():

            # Selecionando canal/aparelho
            # e = self.residencia.elec[e_i]

            # Normalizar nome aparelho/canal de medicao
            aparelho = e.label().lower().replace(" ", "_")

            if aparelho != 'site_meter':
                continue

            try:

                # # Extraindo medicoes de energia da carga
                # power = np.array(e.power_series_all_data(sample_period=taxa_amostral).values[:limite_serie])
                
                # Extraindo medicoes de energia da carga (toda a serie)
                consumo = e.power_series_all_data(sample_period=taxa_amostral)

                # Remover nan (por zero)
                consumo = consumo.replace(np.nan, 0)

                dados.append({
                    "rotulo": aparelho,
                    "instancia": e.instance(),
                    "consumo": consumo
                })

            except Exception as e:
                if debug: print(f"{aparelho}: erro ao extrair dados -> {str(e)}")

    consumo_agregado = pd.concat([c['consumo'] for c in dados], axis=1).sum(1, min_count=1).fillna(0)

    return consumo_agregado

In [15]:
def obter_tipo_aparelho(
    aparelho, 
    tipos = {
        'dish_washer': 'dish washer',
        'fridge': 'fridge',
        'fridge_freezer': 'fridge',
        'microwave': 'microwave',
        'washer_dryer': 'washer_dryer',
        'washing_machine': 'washing_machine'
    }):
    if not any(map(aparelho.__contains__, aparelhos.keys())) or 'site_meter' in aparelho:
        return None
    else:
        return tipos[aparelho]
        

In [16]:
from pathlib import Path
from glob import glob

from PyNILM.dados.utils import *
from PyNILM.modelos.utils import *
from pyts.image import RecurrencePlot

def converter_serie_para_rp(
    serie,
    input_shape=TAMANHO_IMAGEM_DLAFE,
    data_type=np.float32,
    normalize=False, 
    standardize=False, 
    rescale=False,
    # persistir=True,
    # deletar_arquivo_amostra=True
    ):
    """Função de pré-processamento tf.data"""
    # Carregando janelas de consumo (X) e estados da carga (y)
    
    # import os
    # import numpy as np
    # from pathlib import Path
    # from pyts.image import RecurrencePlot
    
    # X = np.load(amostra).astype(data_type)
    # y = np.int8(Path(str(amostra)).stem.split('-')[-1])

    # Transformando janela de consumo em imagem RP
    img = RecurrencePlot(**PARAMETROS_RP).fit_transform([serie])[0]
    img = cv2.resize(
            img, 
            dsize=input_shape[:2], 
            interpolation=cv2.INTER_CUBIC
        ).astype(data_type)

    if np.sum(img) > 0:
        # TODO: improve fit/predict statistics
        # Normalizar
        if normalize:
            img = (img - img.min()) / (img.max() - img.min()) # MinMax (0,1)
            #img = (img - img.mean()) / np.max([img.std(), 1e-4])

        # Padronizar
        elif standardize:
            img = (img - img.mean())/img.std()#tf.image.per_image_standardization(img).numpy()
            
        elif rescale:
            img = (img - img.min()) / (img.max() - img.min())

    # N canais
    X_rp = np.stack([img for _ in range(input_shape[-1])],axis=-1).astype(data_type)  

    return X_rp

In [16]:
# Periodo em dias para o estudo
PERIODO = 60

# Consolidando dados ukdale (pré-treino)
dados_ukdale = carregar_dados(
    ukdale,  
    periodo=PERIODO,
    residencias=None,
    janelas_otimizadas=aparelhos_janelas,  # conhecimento a priori das melhores janelas por aparelho
    # janela_padrao=JANELA_PADRAO, 
    # top_k_consumo=TOP_K,
    ignorar_janelas_vazias=True,
    estatisticas=False,
    metadados=False,
    debug=True
    )
# dados_ukdale = carregar_dados(
#     ukdale,  
#     atraso_inicial=PERIODO, # Contemplar dados nao vistos durante fine-tuning
#     periodo=PERIODO,
#     residencias=None,
#     aparelhos=aparelhos_janelas,
#     estatisticas=False,
#     metadados=False,
#     debug=True
#     )

> Processando residência #1...
* Extraindo consumo agregado dos medidores...
Residencia: Building(instance=1, dataset='UK-DALE')
  - Consumo agregado: de 2012-11-09 22:28:14+00:00 a 2015-01-05 06:27:12+00:00
config_aparelhos {'dish': 2040, 'fridge': 720, 'microwave': 900, 'washer_dryer': 2040, 'washing_machine': 2040}

* Gerando janelas de consumo para cada canal/aparelho...
   - Consumo aparelho washer_dryer: de 2012-11-09 22:28:18+00:00 a 2013-01-08 22:28:18+00:00 | (2592001,) >>> 0.0 16442.965 613.57794 826.2863
  - Consumo agregado/aparelho: de  2012-11-09 22:28:18+00:00 a 2013-01-08 22:28:18+00:00 | (2592001,) >>> 0.0 6368.0 263.10522 393.6171
  - Shapes sincronizados? True
   - Consumo aparelho dish_washer: de 2012-11-09 22:28:18+00:00 a 2013-01-08 22:28:18+00:00 | (2592001,) >>> 0.0 16442.965 613.57794 826.2863
  - Consumo agregado/aparelho: de  2012-11-09 22:28:18+00:00 a 2013-01-08 22:28:18+00:00 | (2592001,) >>> 0.0 6368.0 263.10522 393.6171
  - Shapes sincronizados? True
   

In [17]:
# Exportando base de teste (redd)
# dados_redd = carregar_dados(
#     redd,  
#     periodo=PERIODO,
#     residencias=None,
#     janelas_otimizadas=aparelhos_janelas,  # conhecimento a priori das melhores janelas por aparelho
#     # janela_padrao=JANELA_PADRAO, 
#     top_k_consumo=None,
#     ignorar_janelas_vazias=False,
#     estatisticas=False,
#     metadados=False,
#     debug=True
#     )

# dados_redd = carregar_dados(
#     redd,  
#     atraso_inicial=30,
#     periodo=30,
#     residencias=None,
#     aparelhos=aparelhos_janelas,
#     estatisticas=False,
#     metadados=False,
#     debug=True
#     )

> Processando residência #1...
* Extraindo consumo agregado dos medidores...
Residencia: Building(instance=1, dataset='REDD')
  - Consumo agregado: de 2011-04-18 09:22:08-04:00 a 2011-05-24 15:57:02-04:00
config_aparelhos {'dish': 2040, 'fridge': 720, 'microwave': 900, 'washer_dryer': 2040, 'washing_machine': 2040}

* Gerando janelas de consumo para cada canal/aparelho...
   - Consumo aparelho fridge: de 2011-04-18 09:22:12-04:00 a 2011-05-24 15:56:34-04:00 | (1567032,) >>> 0.0 11870.33 192.70941 584.5038
  - Consumo agregado/aparelho: de  2011-04-18 09:22:12-04:00 a 2011-05-24 15:57:02-04:00 | (1567046,) >>> 0.0 11870.33 192.70923 584.5043
  - Shapes sincronizados? False
   - Consumo aparelho dish_washer: de 2011-04-18 09:22:12-04:00 a 2011-05-24 15:56:34-04:00 | (1567032,) >>> 0.0 11870.33 192.70941 584.5038
  - Consumo agregado/aparelho: de  2011-04-18 09:22:12-04:00 a 2011-05-24 15:57:02-04:00 | (1567046,) >>> 0.0 11870.33 192.70923 584.5043
  - Shapes sincronizados? False
   - Con

In [22]:
## Apagando registros
if dados_ukdale: del dados_ukdale
# if dados_redd: del dados_redd
gc.collect()

56

# Design Experimental
---

Pré-treinar os modelos com base nos dados de UK-DALE e testar com dados REDD.

In [23]:
# for d in dados_ukdale:
#     print(d['aparelho'], d['instancia'], d['janelas'].shape)

In [24]:
for d in dados_redd:
    print(d['aparelho'], d['instancia'], d['janelas'].shape)

fridge 5 (2176, 720)
dish_washer 6 (768, 2040)
microwave 11 (1741, 900)
washer_dryer (10, 20) (768, 2040)
microwave 6 (1668, 900)
washer_dryer 7 (736, 2040)
fridge 9 (2086, 720)
dish_washer 10 (736, 2040)
fridge 7 (2687, 720)
dish_washer 9 (948, 2040)
microwave 16 (2150, 900)
washer_dryer (13, 14) (948, 2040)
washer_dryer 7 (1016, 2040)
dish_washer 15 (1016, 2040)
microwave 3 (2103, 900)
fridge 18 (2629, 720)
dish_washer 20 (928, 2040)
washer_dryer (8, 9) (928, 2040)
washer_dryer 4 (491, 2040)
fridge 8 (1393, 720)
dish_washer 9 (491, 2040)


# Metodologia DLAFE

In [17]:
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost.sklearn import XGBClassifier

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=SEED)

# Teste da classe
janelas_treino = Janelas(
    base=DataSet(arquivo_dataset_redd),
    id_residencia=3,
    inicio_intervalo='2011-04-16 00:00:00',
    fim_intervalo='2011-05-16 23:59:59',
    debug = False
)

janelas_teste = Janelas(
    base=DataSet(arquivo_dataset_redd),
    id_residencia=3,
    inicio_intervalo='2011-05-17 00:00:00',
    fim_intervalo='2011-05-30 23:59:59',
    debug = False
)

aparelhos = [
    'dish_washer - 9',
    'fridge - 7',
    'microwave - 16',
    'washer_dryer - 13', 
    'washer_dryer - 14'
]

TAXA = 2 # Fixa

modelo_extrator = transfer_learning.vgg16.VGG16(
            weights='imagenet', 
            include_top=False,
            pooling='avg'
        )
preprocessamento_extrator = transfer_learning.vgg16.preprocess_input

def carregar_modelo_pretreinado(
    # aparelho,
    caminho_modelo,
    # caminho_modelos_salvos=r'H:\Meu Drive\phd-thesis\datasets\transfer-learning\periodo-60\modelos_salvos',
    debug=False):
    from glob import glob

    # caminho_modelo = glob(os.path.join(caminho_modelos_salvos, f'*{aparelho[:4]}*-final*'))[0]
    if debug: print('Modelo persistido no arquivo:', caminho_modelo)

    modelo = tf.keras.models.load_model(caminho_modelo)
    # return modelo
    return tf.keras.models.Model(
        modelo.input, 
        modelo.layers[-2].output
        )

    # Removendo camadas adicionais (extra-vgg16), incluidas no fine-tuning
    

In [18]:
JANELA = 900

# Listar modelos NILMNET pre-treinados
CAMINHO_MODELOS_SALVOS = f'H:\\Meu Drive\\phd-thesis\\datasets\\transfer-learning\\periodo-60_janela-{JANELA}_taxa-{TAXA}\\nilmnet\\{MODELO_PRETREINADO}'

def modelos_pretreinado_aparelho(aparelho, caminho_modelos_salvos):
    modelos_nilmnet = glob(os.path.join(CAMINHO_MODELOS_SALVOS, f'*{aparelho[:4]}*.ckpt')) + \
        glob(os.path.join(CAMINHO_MODELOS_SALVOS, f'*global*.ckpt'))
    return modelos_nilmnet

### Teste de carregamento de modelo pré-treinado

In [19]:
for aparelho in aparelhos:
    print(aparelho)
    modelos_nilmnet = modelos_pretreinado_aparelho(aparelho, CAMINHO_MODELOS_SALVOS)
    print(modelos_nilmnet)
    print()

dish_washer - 9
['H:\\Meu Drive\\phd-thesis\\datasets\\transfer-learning\\periodo-60_janela-900_taxa-2\\nilmnet\\VGG16\\dish-first_step-20220807144957.ckpt', 'H:\\Meu Drive\\phd-thesis\\datasets\\transfer-learning\\periodo-60_janela-900_taxa-2\\nilmnet\\VGG16\\global-first_step-20220807150150.ckpt']

fridge - 7
['H:\\Meu Drive\\phd-thesis\\datasets\\transfer-learning\\periodo-60_janela-900_taxa-2\\nilmnet\\VGG16\\fridge-first_step-20220807142200.ckpt', 'H:\\Meu Drive\\phd-thesis\\datasets\\transfer-learning\\periodo-60_janela-900_taxa-2\\nilmnet\\VGG16\\global-first_step-20220807150150.ckpt']

microwave - 16
['H:\\Meu Drive\\phd-thesis\\datasets\\transfer-learning\\periodo-60_janela-900_taxa-2\\nilmnet\\VGG16\\microwave-first_step-20220807144224.ckpt', 'H:\\Meu Drive\\phd-thesis\\datasets\\transfer-learning\\periodo-60_janela-900_taxa-2\\nilmnet\\VGG16\\global-first_step-20220807150150.ckpt']

washer_dryer - 13
['H:\\Meu Drive\\phd-thesis\\datasets\\transfer-learning\\periodo-60_janela

## SVM

In [20]:
modelo = SVC(kernel='rbf', random_state=SEED)

resultados_modelo = {
    "appliance": [], "fold": [],
    "acc": [], "f1": [], "auc": [], 
    "base": [], "nilmnet": []
}

for rotulo_aparelho in aparelhos:
    
    print(f"* Aparelho `{rotulo_aparelho}`...\n")
    
    # Informacoes da carga selecionada
    CARGA = rotulo_aparelho.split(" - ")[0]
    INSTANCIA = int(rotulo_aparelho.split(" - ")[1])
    config_aparelho = df_melhores_taxas_janelas[
        df_melhores_taxas_janelas["carga"]==rotulo_aparelho
    ].to_dict("records")[0]
    TAMANHO_JANELA = config_aparelho["janela"]

    # Instanciando modelos pretreinados relativos a carga + global
    modelos_nilmnet = modelos_pretreinado_aparelho(rotulo_aparelho, CAMINHO_MODELOS_SALVOS)

    for caminho_modelo_nilmnet in modelos_nilmnet:

        nome_modelo_nilmnet = f"{MODELO_PRETREINADO}_{Path(caminho_modelo_nilmnet).stem.split('_')[0]}"

        print(f"   - Carregando modelo pré-treinado `{nome_modelo_nilmnet}`...\n")
        modelo_extrator = carregar_modelo_pretreinado(caminho_modelo_nilmnet)

        #######################################################################
        #                AVALIACAO 1 - Base de treino / CV                    #
        #######################################################################
        # Extrair series divididas em janelas para cada medidor
        print("   - Base de TREINO\n")
        print("     -> Carregando dados (taxa={:.0f}, janela={:.0f})...".format(
            TAXA, TAMANHO_JANELA
        ))
        X, y = carregar_dados_aparelho(
            janelas=janelas_treino,
            instancia=INSTANCIA,
            aparelho=CARGA,
            tamanho_janela=TAMANHO_JANELA,
            taxa=TAXA,
            eliminar_janelas_vazias=True
        )
        print("     -> Detalhes da amostragem (lotes):")
        print("     ---")
        for item in Counter(y).items():
            print(f"        - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y)*100,1)}%)" )
        print()
        
        y_true, y_pred  = [], []

        print(f"     -> Avaliando modelo (CV - {skf.n_splits} folds)...\n")
        for it, (idx_treino, idx_teste) in tqdm_notebook(enumerate(skf.split(X, y)), total=skf.n_splits):

            # Preparando lotes
            X_treino, X_teste = X[idx_treino], X[idx_teste]
            y_treino, y_teste = y[idx_treino], y[idx_teste]

            # Treinando modelo
            dlafe = DLAFE(
                feature_extractor=modelo_extrator,
                preprocess_input=preprocessamento_extrator,
                classifier=clone(modelo),
                rp_params = PARAMETROS_RP,
                input_shape = modelo_extrator.input_shape[1:], # TAMANHO_IMAGEM_DLAFE,
                normalize=False
            )
            dlafe.fit(X_treino, y_treino)

            # Prevendo conjunto de teste
            y_hat = dlafe.predict(X_teste)

            # Incrementando resultados
            resultados_modelo["appliance"].append(rotulo_aparelho)
            resultados_modelo["fold"].append(it+1)
            resultados_modelo["acc"].append( accuracy_score(y_teste, y_hat) )
            resultados_modelo["f1"].append( f1_score(y_teste, y_hat, average="macro") )
            resultados_modelo["auc"].append(roc_auc_score(y_teste, y_hat) if np.unique(y_teste).shape[0]>1 else 0.5)
            resultados_modelo["base"].append("treino")
            resultados_modelo["nilmnet"].append(nome_modelo_nilmnet)
            
            reset_tf_session(model_name='dlafe')

            # Extendendo rotulos (analise global)
            y_true.extend(y_teste)
            y_pred.extend(y_hat)
            
        #######################################################################
        #                 AVALIACAO 2 - Base de teste / CV                    #
        #######################################################################
        print("   - Base de TESTE\n")
        print("     -> Carregando dados (taxa={:.0f}, janela={:.0f})...".format(
            TAXA, TAMANHO_JANELA
        ))

        # Avaliar na base de teste
        X_teste, y_teste = carregar_dados_aparelho(
            janelas=janelas_teste,
            instancia=INSTANCIA,
            aparelho=CARGA,
            tamanho_janela=TAMANHO_JANELA,
            taxa=TAXA,
            eliminar_janelas_vazias=True
        )
        
        print("     -> Detalhes da amostragem (lotes):")
        print("     ---")
        for item in Counter(y_teste).items():
            print(f"       - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y_teste)*100,1)}%)" )
        print()

        # Treinando modelo
        dlafe = DLAFE(
            feature_extractor=modelo_extrator,
            preprocess_input=preprocessamento_extrator,
            classifier=clone(modelo),
            rp_params = PARAMETROS_RP,
            input_shape = modelo_extrator.input_shape[1:],
            normalize=False
        )
        dlafe.fit(X, y)

        # Prevendo conjunto de teste
        y_hat = dlafe.predict(X_teste)

        # Incrementando resultados
        resultados_modelo["appliance"].append(rotulo_aparelho)
        resultados_modelo["fold"].append(it+1)
        resultados_modelo["acc"].append( accuracy_score(y_teste, y_hat) )
        resultados_modelo["f1"].append( f1_score(y_teste, y_hat, average="macro") )
        resultados_modelo["auc"].append(roc_auc_score(y_teste, y_hat) if np.unique(y_teste).shape[0]>1 else 0.5)
        resultados_modelo["base"].append("teste")
        resultados_modelo["nilmnet"].append(nome_modelo_nilmnet)
        
        reset_tf_session(model_name='dlafe')
        
        print()
        print("   - Final Results:")
        print("   ---")
        print()
        
        print("***** TRAIN *****")
        print("      -> Classification Report:")
        print()
        print(classification_report(y_true, y_pred))
        print("      -> Confusion Matrix:")
        print()
        print(confusion_matrix(y_true, y_pred))
        print()
        
        print("***** TEST *****")
        print("      -> Classification Report:")
        print()
        print(classification_report(y_teste, y_hat))
        print("      -> Confusion Matrix:")
        print()
        print(confusion_matrix(y_teste, y_hat))
        print()
    
# Consolidating DataFrame
df_resultados = pd.DataFrame(resultados_modelo)
df_resultados.to_excel(os.path.join(caminho_dados_notebook, "df_resultados_svm.xlsx"))
    
print("############################## FINAL MODEL RESULTS ##############################")
display(df_resultados.groupby(["appliance","base"]).agg({
    "acc": ["mean", "std", "max", "min"],
    "f1": ["mean", "std", "max", "min"],
    "auc": ["mean", "std", "max", "min"]
}))

* Aparelho `dish_washer - 9`...

   - Carregando modelo pré-treinado `VGG16_dish-first`...

   - Base de TREINO

     -> Carregando dados (taxa=2, janela=2040)...
Meter 13 is in a nested meter group. Retrieving just the ElecMeter.
Meter 14 is in a nested meter group. Retrieving just the ElecMeter.

     -> Detalhes da amostragem (lotes):
     ---
        - Classe `0`: 217 amostras (94.8%)
        - Classe `1`: 12 amostras (5.2%)

     -> Avaliando modelo (CV - 10 folds)...



  0%|          | 0/10 [00:00<?, ?it/s]

Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
   - Base de TESTE

     -> Carregando dados (taxa=2, janela=2040)...
Meter 13 is in a nested meter group. Retrieving just the ElecMeter.
Meter 14 is in a nested meter group. Retrieving just the ElecMeter.

     -> Detalhes da amostragem (lotes):
     ---
       - Classe `0`: 138 amostras (93.2%)
       - Classe `1`: 10 amostras (6.8%)

Virtual devices cannot be modified after being initialized

   - Final 

  0%|          | 0/10 [00:00<?, ?it/s]

Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
   - Base de TESTE

     -> Carregando dados (taxa=2, janela=2040)...
Meter 13 is in a nested meter group. Retrieving just the ElecMeter.
Meter 14 is in a nested meter group. Retrieving just the ElecMeter.

     -> Detalhes da amostragem (lotes):
     ---
       - Classe `0`: 138 amostras (93.2%)
       - Classe `1`: 10 amostras (6.8%)

Virtual devices cannot be modified after being initialized

   - Final 

...

  0%|          | 0/10 [00:00<?, ?it/s]

Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
   - Base de TESTE

     -> Carregando dados (taxa=2, janela=2040)...
Meter 13 is in a nested meter group. Retrieving just the ElecMeter.
Meter 14 is in a nested meter group. Retrieving just the ElecMeter.

     -> Detalhes da amostragem (lotes):
     ---
       - Classe `0`: 134 amostras (90.5%)
       - Classe `1`: 14 amostras (9.5%)

Virtual devices cannot be modified after being initialized

   - Final 

  0%|          | 0/10 [00:00<?, ?it/s]

Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
   - Base de TESTE

     -> Carregando dados (taxa=2, janela=2040)...
Meter 13 is in a nested meter group. Retrieving just the ElecMeter.
Meter 14 is in a nested meter group. Retrieving just the ElecMeter.

     -> Detalhes da amostragem (lotes):
     ---
       - Classe `0`: 134 amostras (90.5%)
       - Classe `1`: 14 amostras (9.5%)

Virtual devices cannot be modified after being initialized

   - Final 

acc                                      f1  \
                              mean       std       max       min      mean   
appliance         base                                                       
dish_washer - 9   teste   0.932432  0.000000  0.932432  0.932432  0.482517   
                  treino  0.947628  0.017752  0.956522  0.913043  0.486514   
fridge - 7        teste   0.956098  0.000000  0.956098  0.956098  0.488778   
                  treino  0.926463  0.007196  0.937500  0.921875  0.480907   
microwave - 16    teste   0.939024  0.000000  0.939024  0.939024  0.484277   
                  treino  0.945211  0.007996  0.960784  0.941176  0.485909   
washer_dryer - 13 teste   0.989487  0.010058  0.996599  0.982375  0.926015   
                  treino  0.990964  0.007274  1.000000  0.980119  0.898324   
washer_dryer - 14 teste   0.908784  0.004778  0.912162  0.905405  0.534763   
                  treino  0.936759  0.022016  0.956522  0.913043  0.529337   

                                                             auc            \
                               std       max       min      mean       std   
appliance         base                                                       
dish_washer - 9   teste   0.000000  0.482517  0.482517  0.500000  0.000000   
                  treino  0.004743  0.488889  0.477273  0.500000  0.000000   
fridge - 7        teste   0.000000  0.488778  0.488778  0.500000  0.000000   
                  treino  0.001933  0.483871  0.479675  0.500000  0.000000   
microwave - 16    teste   0.000000  0.484277  0.484277  0.500000  0.000000   
                  treino  0.002101  0.490000  0.484848  0.500000  0.000000   
washer_dryer - 13 teste   0.073347  0.977879  0.874151  0.902039  0.094999   
                  treino  0.085035  1.000000  0.730753  0.872901  0.102381   
washer_dryer - 14 teste   0.084268  0.594350  0.475177  0.533849  0.047869   
                  treino  0.113834  0.821705  0.477273  0.547673  0.124815   

                                              
                               max       min  
appliance         base                        
dish_washer - 9   teste   0.500000  0.500000  
                  treino  0.500000  0.500000  
fridge - 7        teste   0.500000  0.500000  
                  treino  0.500000  0.500000  
microwave - 16    teste   0.500000  0.500000  
                  treino  0.500000  0.500000  
washer_dryer - 13 teste   0.969213  0.834865  
                  treino  1.000000  0.653846  
washer_dryer - 14 teste   0.567697  0.500000  
                  treino  0.977273  0.500000

## XGBOOST

In [21]:
def pos_weight(y):
    try:
        counter = Counter(y)
        return counter[0]/counter[1]
    except:
        return 1

In [22]:
modelo = XGBClassifier(random_state=SEED, n_jobs=4)

resultados_modelo = {
    "appliance": [], "fold": [],
    "acc": [], "f1": [], "auc": [], 
    "base": [], "nilmnet": []
}

for rotulo_aparelho in aparelhos:
    
    print(f"* Aparelho `{rotulo_aparelho}`...\n")
    
    # Informacoes da carga selecionada
    CARGA = rotulo_aparelho.split(" - ")[0]
    INSTANCIA = int(rotulo_aparelho.split(" - ")[1])
    config_aparelho = df_melhores_taxas_janelas[
        df_melhores_taxas_janelas["carga"]==rotulo_aparelho
    ].to_dict("records")[0]
    TAMANHO_JANELA = config_aparelho["janela"]

    # Instanciando modelos pretreinados relativos a carga + global
    modelos_nilmnet = modelos_pretreinado_aparelho(rotulo_aparelho, CAMINHO_MODELOS_SALVOS)

    for caminho_modelo_nilmnet in modelos_nilmnet:

        nome_modelo_nilmnet = f"{MODELO_PRETREINADO}_{Path(caminho_modelo_nilmnet).stem.split('_')[0]}"

        print(f"   - Carregando modelo pré-treinado `{nome_modelo_nilmnet}`...\n")
        modelo_extrator = carregar_modelo_pretreinado(caminho_modelo_nilmnet)

        #######################################################################
        #                AVALIACAO 1 - Base de treino / CV                    #
        #######################################################################
        # Extrair series divididas em janelas para cada medidor
        print("   - Base de TREINO\n")
        print("     -> Carregando dados (taxa={:.0f}, janela={:.0f})...".format(
            TAXA, TAMANHO_JANELA
        ))
        X, y = carregar_dados_aparelho(
            janelas=janelas_treino,
            instancia=INSTANCIA,
            aparelho=CARGA,
            tamanho_janela=TAMANHO_JANELA,
            taxa=TAXA,
            eliminar_janelas_vazias=True
        )
        print("     -> Detalhes da amostragem (lotes):")
        print("     ---")
        for item in Counter(y).items():
            print(f"        - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y)*100,1)}%)" )
        print()
        
        y_true, y_pred  = [], []

        print(f"     -> Avaliando modelo (CV - {skf.n_splits} folds)...\n")
        for it, (idx_treino, idx_teste) in tqdm_notebook(enumerate(skf.split(X, y)), total=skf.n_splits):

            # Preparando lotes
            X_treino, X_teste = X[idx_treino], X[idx_teste]
            y_treino, y_teste = y[idx_treino], y[idx_teste]

            # Treinando modelo
            dlafe = DLAFE(
                feature_extractor=modelo_extrator,
                preprocess_input=preprocessamento_extrator,
                classifier=clone(modelo),#.set_params(**{'scale_pos_weight': pos_weight(y)}),
                rp_params = PARAMETROS_RP,
                input_shape = modelo_extrator.input_shape[1:], #TAMANHO_IMAGEM_DLAFE,
                normalize=False
            )
            dlafe.fit(X_treino, y_treino)

            # Prevendo conjunto de teste
            y_hat = dlafe.predict(X_teste)

            # Incrementando resultados
            resultados_modelo["appliance"].append(rotulo_aparelho)
            resultados_modelo["fold"].append(it+1)
            resultados_modelo["acc"].append( accuracy_score(y_teste, y_hat) )
            resultados_modelo["f1"].append( f1_score(y_teste, y_hat, average="macro") )
            resultados_modelo["auc"].append(roc_auc_score(y_teste, y_hat) if np.unique(y_teste).shape[0]>1 else 0.5)
            resultados_modelo["base"].append("treino")
            resultados_modelo["nilmnet"].append(nome_modelo_nilmnet)
            
            reset_tf_session(model_name='dlafe')

            # Extendendo rotulos (analise global)
            y_true.extend(y_teste)
            y_pred.extend(y_hat)
            
        #######################################################################
        #                 AVALIACAO 2 - Base de teste / CV                    #
        #######################################################################
        print("   - Base de TESTE\n")
        print("     -> Carregando dados (taxa={:.0f}, janela={:.0f})...".format(
            TAXA, TAMANHO_JANELA
        ))

        # Avaliar na base de teste
        X_teste, y_teste = carregar_dados_aparelho(
            janelas=janelas_teste,
            instancia=INSTANCIA,
            aparelho=CARGA,
            tamanho_janela=TAMANHO_JANELA,
            taxa=TAXA,
            eliminar_janelas_vazias=True
        )
        
        print("     -> Detalhes da amostragem (lotes):")
        print("     ---")
        for item in Counter(y_teste).items():
            print(f"       - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y_teste)*100,1)}%)" )
        print()

        # Treinando modelo
        dlafe = DLAFE(
            feature_extractor=modelo_extrator,
            preprocess_input=preprocessamento_extrator,
            classifier=clone(modelo),#.set_params(**{'scale_pos_weight': pos_weight(y)}),
            rp_params = PARAMETROS_RP,
            input_shape = modelo_extrator.input_shape[1:],
            normalize=False
        )
        dlafe.fit(X, y)

        # Prevendo conjunto de teste
        y_hat = dlafe.predict(X_teste)

        # Incrementando resultados
        resultados_modelo["appliance"].append(rotulo_aparelho)
        resultados_modelo["fold"].append(it+1)
        resultados_modelo["acc"].append( accuracy_score(y_teste, y_hat) )
        resultados_modelo["f1"].append( f1_score(y_teste, y_hat, average="macro") )
        resultados_modelo["auc"].append(roc_auc_score(y_teste, y_hat) if np.unique(y_teste).shape[0]>1 else 0.5)
        resultados_modelo["base"].append("teste")
        resultados_modelo["nilmnet"].append(nome_modelo_nilmnet)
        
        reset_tf_session(model_name='dlafe')
        
        print()
        print("   - Final Results:")
        print("   ---")
        print()
        
        print("***** TRAIN *****")
        print("      -> Classification Report:")
        print()
        print(classification_report(y_true, y_pred))
        print("      -> Confusion Matrix:")
        print()
        print(confusion_matrix(y_true, y_pred))
        print()
        
        print("***** TEST *****")
        print("      -> Classification Report:")
        print()
        print(classification_report(y_teste, y_hat))
        print("      -> Confusion Matrix:")
        print()
        print(confusion_matrix(y_teste, y_hat))
        print()
        
    
# Consolidating DataFrame
df_resultados = pd.DataFrame(resultados_modelo)
df_resultados.to_excel(os.path.join(caminho_dados_notebook, "df_resultados_xgboost.xlsx"))
    
print("############################## FINAL MODEL RESULTS ##############################")
display(df_resultados.groupby(["appliance","base"]).agg({
    "acc": ["mean", "std", "max", "min"],
    "f1": ["mean", "std", "max", "min"],
    "auc": ["mean", "std", "max", "min"]
}))

* Aparelho `dish_washer - 9`...

   - Carregando modelo pré-treinado `VGG16_dish-first`...

   - Base de TREINO

     -> Carregando dados (taxa=2, janela=2040)...
Meter 13 is in a nested meter group. Retrieving just the ElecMeter.
Meter 14 is in a nested meter group. Retrieving just the ElecMeter.

     -> Detalhes da amostragem (lotes):
     ---
        - Classe `0`: 217 amostras (94.8%)
        - Classe `1`: 12 amostras (5.2%)

     -> Avaliando modelo (CV - 10 folds)...



  0%|          | 0/10 [00:00<?, ?it/s]

[21:56:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Virtual devices cannot be modified after being initialized
[21:56:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Virtual devices cannot be modified after being initialized
[21:56:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_me

  0%|          | 0/10 [00:00<?, ?it/s]

[21:58:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Virtual devices cannot be modified after being initialized
[21:58:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Virtual devices cannot be modified after being initialized
[21:58:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_me

...

  0%|          | 0/10 [00:00<?, ?it/s]

[22:08:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Virtual devices cannot be modified after being initialized
[22:08:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Virtual devices cannot be modified after being initialized
[22:08:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_me

  0%|          | 0/10 [00:00<?, ?it/s]

[22:10:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Virtual devices cannot be modified after being initialized
[22:10:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Virtual devices cannot be modified after being initialized
[22:10:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_me

acc                                      f1  \
                              mean       std       max       min      mean   
appliance         base                                                       
dish_washer - 9   teste   0.908784  0.023889  0.925676  0.891892  0.476065   
                  treino  0.930138  0.035725  1.000000  0.869565  0.523374   
fridge - 7        teste   0.960976  0.006899  0.965854  0.956098  0.642703   
                  treino  0.941307  0.020221  0.984375  0.921875  0.652334   
microwave - 16    teste   0.935976  0.012935  0.945122  0.926829  0.634004   
                  treino  0.933484  0.023212  0.980392  0.882353  0.598779   
washer_dryer - 13 teste   0.993506  0.004810  0.996908  0.990105  0.958069   
                  treino  0.992950  0.006361  1.000000  0.980159  0.927395   
washer_dryer - 14 teste   0.962838  0.023889  0.979730  0.945946  0.869024   
                  treino  0.960771  0.031243  1.000000  0.913043  0.837781   

                                                             auc            \
                               std       max       min      mean       std   
appliance         base                                                       
dish_washer - 9   teste   0.006557  0.480702  0.471429  0.487319  0.012810   
                  treino  0.135895  1.000000  0.465116  0.526028  0.126965   
fridge - 7        teste   0.217682  0.796627  0.488778  0.648313  0.209747   
                  treino  0.181832  0.940243  0.479675  0.650784  0.164952   
microwave - 16    teste   0.162055  0.748595  0.519414  0.626948  0.155655   
                  treino  0.139710  0.894845  0.474227  0.599607  0.125622   
washer_dryer - 13 teste   0.031375  0.980255  0.935884  0.954908  0.035845   
                  treino  0.063961  1.000000  0.828221  0.913098  0.071935   
washer_dryer - 14 teste   0.092547  0.934465  0.803583  0.819563  0.103654   
                  treino  0.153866  1.000000  0.477273  0.877056  0.166699   

                                              
                               max       min  
appliance         base                        
dish_washer - 9   teste   0.496377  0.478261  
                  treino  1.000000  0.454545  
fridge - 7        teste   0.796627  0.500000  
                  treino  0.983333  0.500000  
microwave - 16    teste   0.737013  0.516883  
                  treino  0.833333  0.479167  
washer_dryer - 13 teste   0.980255  0.929562  
                  treino  1.000000  0.784694  
washer_dryer - 14 teste   0.892857  0.746269  
                  treino  1.000000  0.500000

## MLP

In [23]:
def class_weight(y, debug=False):
    
    # Classes distribution
    neg, pos = np.bincount(y)
    total = neg + pos

    # Scaling by total/2 helps keep the loss to a similar magnitude.
    # The sum of the weights of all examples stays the same.
    w_0 = (1 / neg)*(total)/2.0 
    w_1 = (1 / pos)*(total)/2.0

    class_weight = {0: w_0, 1: w_1}

    print('Weight for class 0: {:.2f}'.format(w_0))
    print('Weight for class 1: {:.2f}'.format(w_1))
    
    return class_weight

In [24]:
def mlp(
    input_shape=TAMANHO_IMAGEM_DLAFE,
    metrics=[
        tf.keras.metrics.TruePositives(name='tp'),
        tf.keras.metrics.FalsePositives(name='fp'),
        tf.keras.metrics.TrueNegatives(name='tn'),
        tf.keras.metrics.FalseNegatives(name='fn'), 
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc'),
        tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
    ], 
    output_bias=None
):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
        
    model = tf.keras.Sequential([
      tf.keras.layers.Dense(10, activation='relu', input_shape=input_shape),
      # keras.layers.Dropout(0.5),
      tf.keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias),
    ])

    model.compile(
      optimizer=tf.keras.optimizers.Adam(lr=0.001),
      loss=tf.keras.losses.BinaryCrossentropy(),
      metrics=metrics
    )

    return model

In [25]:
modelo = MLPClassifier(alpha=1e-3, hidden_layer_sizes=(10,), random_state=SEED)

resultados_modelo = {
    "appliance": [], "fold": [],
    "acc": [], "f1": [], "auc": [], 
    "base": [], "nilmnet": []
}

for rotulo_aparelho in aparelhos:
    
    print(f"* Aparelho `{rotulo_aparelho}`...\n")
    
    # Informacoes da carga selecionada
    CARGA = rotulo_aparelho.split(" - ")[0]
    INSTANCIA = int(rotulo_aparelho.split(" - ")[1])
    config_aparelho = df_melhores_taxas_janelas[
        df_melhores_taxas_janelas["carga"]==rotulo_aparelho
    ].to_dict("records")[0]
    TAMANHO_JANELA = config_aparelho["janela"]

    # Instanciando modelos pretreinados relativos a carga + global
    modelos_nilmnet = modelos_pretreinado_aparelho(rotulo_aparelho, CAMINHO_MODELOS_SALVOS)

    for caminho_modelo_nilmnet in modelos_nilmnet:

        nome_modelo_nilmnet = f"{MODELO_PRETREINADO}_{Path(caminho_modelo_nilmnet).stem.split('_')[0]}"

        print(f"   - Carregando modelo pré-treinado `{nome_modelo_nilmnet}`...\n")
        modelo_extrator = carregar_modelo_pretreinado(caminho_modelo_nilmnet)

        #######################################################################
        #                AVALIACAO 1 - Base de treino / CV                    #
        #######################################################################
        # Extrair series divididas em janelas para cada medidor
        print("   - Base de TREINO\n")
        print("     -> Carregando dados (taxa={:.0f}, janela={:.0f})...".format(
            TAXA, TAMANHO_JANELA
        ))
        X, y = carregar_dados_aparelho(
            janelas=janelas_treino,
            instancia=INSTANCIA,
            aparelho=CARGA,
            tamanho_janela=TAMANHO_JANELA,
            taxa=TAXA,
            eliminar_janelas_vazias=True
        )
        print("     -> Detalhes da amostragem (lotes):")
        print("     ---")
        for item in Counter(y).items():
            print(f"        - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y)*100,1)}%)" )
        print()
        
        y_true, y_pred  = [], []

        print(f"     -> Avaliando modelo (CV - {skf.n_splits} folds)...\n")
        for it, (idx_treino, idx_teste) in tqdm_notebook(enumerate(skf.split(X, y)), total=skf.n_splits):

            # Preparando lotes
            X_treino, X_teste = X[idx_treino], X[idx_teste]
            y_treino, y_teste = y[idx_treino], y[idx_teste]

            # Treinando modelo
            dlafe = DLAFE(
                feature_extractor=modelo_extrator,
                preprocess_input=preprocessamento_extrator,
                classifier=clone(modelo),
                rp_params = PARAMETROS_RP,
                input_shape = modelo_extrator.input_shape[1:],
                normalize=False
            )
            dlafe.fit(X_treino, y_treino)

            # Prevendo conjunto de teste
            y_hat = dlafe.predict(X_teste)

            # Incrementando resultados
            resultados_modelo["appliance"].append(rotulo_aparelho)
            resultados_modelo["fold"].append(it+1)
            resultados_modelo["acc"].append( accuracy_score(y_teste, y_hat) )
            resultados_modelo["f1"].append( f1_score(y_teste, y_hat, average="macro") )
            resultados_modelo["auc"].append(roc_auc_score(y_teste, y_hat) if np.unique(y_teste).shape[0]>1 else 0.5)
            resultados_modelo["base"].append("treino")
            resultados_modelo["nilmnet"].append(nome_modelo_nilmnet)
            
            reset_tf_session(model_name='dlafe')

            # Extendendo rotulos (analise global)
            y_true.extend(y_teste)
            y_pred.extend(y_hat)
            
        #######################################################################
        #                 AVALIACAO 2 - Base de teste / CV                    #
        #######################################################################
        print("   - Base de TESTE\n")
        print("     -> Carregando dados (taxa={:.0f}, janela={:.0f})...".format(
            TAXA, TAMANHO_JANELA
        ))

        # Avaliar na base de teste
        X_teste, y_teste = carregar_dados_aparelho(
            janelas=janelas_teste,
            instancia=INSTANCIA,
            aparelho=CARGA,
            tamanho_janela=TAMANHO_JANELA,
            taxa=TAXA,
            eliminar_janelas_vazias=True
        )
        
        print("     -> Detalhes da amostragem (lotes):")
        print("     ---")
        for item in Counter(y_teste).items():
            print(f"       - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y)*100,1)}%)" )
        print()

        # Treinando modelo
        dlafe = DLAFE(
            feature_extractor=modelo_extrator,
            preprocess_input=preprocessamento_extrator,
            classifier=clone(modelo),
            rp_params = PARAMETROS_RP,
            input_shape = modelo_extrator.input_shape[1:],
            normalize=False
        )
        dlafe.fit(X, y)

        # Prevendo conjunto de teste
        y_hat = dlafe.predict(X_teste)

        # Incrementando resultados
        resultados_modelo["appliance"].append(rotulo_aparelho)
        resultados_modelo["fold"].append(it+1)
        resultados_modelo["acc"].append( accuracy_score(y_teste, y_hat) )
        resultados_modelo["f1"].append( f1_score(y_teste, y_hat, average="macro") )
        resultados_modelo["auc"].append(roc_auc_score(y_teste, y_hat) if np.unique(y_teste).shape[0]>1 else 0.5)
        resultados_modelo["base"].append("teste")
        resultados_modelo["nilmnet"].append(nome_modelo_nilmnet)
        
        reset_tf_session(model_name='dlafe')
        
        print()
        print("   - Final Results:")
        print("   ---")
        print()
        
        print("***** TRAIN *****")
        print("      -> Classification Report:")
        print()
        print(classification_report(y_true, y_pred))
        print("      -> Confusion Matrix:")
        print()
        print(confusion_matrix(y_true, y_pred))
        print()
        
        print("***** TEST *****")
        print("      -> Classification Report:")
        print()
        print(classification_report(y_teste, y_hat))
        print("      -> Confusion Matrix:")
        print()
        print(confusion_matrix(y_teste, y_hat))
        print()
    
# Consolidating DataFrame
df_resultados = pd.DataFrame(resultados_modelo)
df_resultados.to_excel(os.path.join(caminho_dados_notebook, "df_resultados_mlp.xlsx"))
    
print("############################## FINAL MODEL RESULTS ##############################")
display(df_resultados.groupby(["appliance","base"]).agg({
    "acc": ["mean", "std", "max", "min"],
    "f1": ["mean", "std", "max", "min"],
    "auc": ["mean", "std", "max", "min"]
}))

* Aparelho `dish_washer - 9`...

   - Carregando modelo pré-treinado `VGG16_dish-first`...

   - Base de TREINO

     -> Carregando dados (taxa=2, janela=2040)...
Meter 13 is in a nested meter group. Retrieving just the ElecMeter.
Meter 14 is in a nested meter group. Retrieving just the ElecMeter.

     -> Detalhes da amostragem (lotes):
     ---
        - Classe `0`: 217 amostras (94.8%)
        - Classe `1`: 12 amostras (5.2%)

     -> Avaliando modelo (CV - 10 folds)...



  0%|          | 0/10 [00:00<?, ?it/s]

Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
   - Base de TESTE

     -> Carregando dados (taxa=2, janela=2040)...
Meter 13 is in a nested meter group. Retrieving just the ElecMeter.
Meter 14 is in a nested meter group. Retrieving just the ElecMeter.

     -> Detalhes da amostragem (lotes):
     ---
       - Classe `0`: 138 amostras (60.3%)
       - Classe `1`: 10 amostras (4.4%)

Virtual devices cannot be modified after being initialized

   - Final 

  0%|          | 0/10 [00:00<?, ?it/s]

Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
   - Base de TESTE

     -> Carregando dados (taxa=2, janela=2040)...
Meter 13 is in a nested meter group. Retrieving just the ElecMeter.
Meter 14 is in a nested meter group. Retrieving just the ElecMeter.

     -> Detalhes da amostragem (lotes):
     ---
       - Classe `0`: 138 amostras (60.3%)
       - Classe `1`: 10 amostras (4.4%)

Virtual devices cannot be modified after being initialized

   - Final 

...

  0%|          | 0/10 [00:00<?, ?it/s]

Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
   - Base de TESTE

     -> Carregando dados (taxa=2, janela=2040)...
Meter 13 is in a nested meter group. Retrieving just the ElecMeter.
Meter 14 is in a nested meter group. Retrieving just the ElecMeter.

     -> Detalhes da amostragem (lotes):
     ---
       - Classe `0`: 134 amostras (58.5%)
       - Classe `1`: 14 amostras (6.1%)

Virtual devices cannot be modified after being initialized

   - Final 

  0%|          | 0/10 [00:00<?, ?it/s]

Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
   - Base de TESTE

     -> Carregando dados (taxa=2, janela=2040)...
Meter 13 is in a nested meter group. Retrieving just the ElecMeter.
Meter 14 is in a nested meter group. Retrieving just the ElecMeter.

     -> Detalhes da amostragem (lotes):
     ---
       - Classe `0`: 134 amostras (58.5%)
       - Classe `1`: 14 amostras (6.1%)

Virtual devices cannot be modified after being initialized

   - Final 

acc                                      f1  \
                              mean       std       max       min      mean   
appliance         base                                                       
dish_washer - 9   teste   0.932432  0.000000  0.932432  0.932432  0.482517   
                  treino  0.947628  0.017752  0.956522  0.913043  0.486514   
fridge - 7        teste   0.954878  0.001725  0.956098  0.953659  0.488459   
                  treino  0.924120  0.010353  0.937500  0.906250  0.488588   
microwave - 16    teste   0.939024  0.000000  0.939024  0.939024  0.484277   
                  treino  0.944250  0.009397  0.960784  0.923077  0.485651   
washer_dryer - 13 teste   0.992579  0.006122  0.996908  0.988250  0.952661   
                  treino  0.992751  0.005324  1.000000  0.984095  0.916940   
washer_dryer - 14 teste   0.945946  0.047777  0.979730  0.912162  0.739000   
                  treino  0.917194  0.147152  1.000000  0.304348  0.602112   

                                                             auc            \
                               std       max       min      mean       std   
appliance         base                                                       
dish_washer - 9   teste   0.000000  0.482517  0.482517  0.500000  0.000000   
                  treino  0.004743  0.488889  0.477273  0.500000  0.000000   
fridge - 7        teste   0.000451  0.488778  0.488140  0.499362  0.000902   
                  treino  0.038150  0.650273  0.475410  0.503312  0.023017   
microwave - 16    teste   0.000000  0.484277  0.484277  0.500000  0.000000   
                  treino  0.002480  0.490000  0.480000  0.499490  0.002282   
washer_dryer - 13 teste   0.038401  0.979815  0.925508  0.948985  0.028836   
                  treino  0.070491  1.000000  0.773729  0.892403  0.097498   
washer_dryer - 14 teste   0.276429  0.934465  0.543535  0.714286  0.252538   
                  treino  0.217530  1.000000  0.292308  0.618452  0.187932   

                                              
                               max       min  
appliance         base                        
dish_washer - 9   teste   0.500000  0.500000  
                  treino  0.500000  0.500000  
fridge - 7        teste   0.500000  0.498724  
                  treino  0.600000  0.491525  
microwave - 16    teste   0.500000  0.500000  
                  treino  0.500000  0.489796  
washer_dryer - 13 teste   0.969375  0.928594  
                  treino  1.000000  0.692308  
washer_dryer - 14 teste   0.892857  0.535714  
                  treino  1.000000  0.500000

# Análise dos Resultados (1)

In [26]:
df_resultados_svm = pd.read_excel(os.path.join(caminho_dados_notebook, "df_resultados_svm.xlsx"), engine="openpyxl")
df_resultados_svm["model"] = "SVM"

df_resultados_xgboost = pd.read_excel(os.path.join(caminho_dados_notebook, "df_resultados_xgboost.xlsx"), engine="openpyxl")
df_resultados_xgboost["model"] = "XGBOOST"

df_resultados_mlp = pd.read_excel(os.path.join(caminho_dados_notebook, "df_resultados_mlp.xlsx"), engine="openpyxl")
df_resultados_mlp["model"] = "MLP"

# df_resultados_elm = pd.read_excel(os.path.join(caminho_dados_notebook, "df_resultados_elm.xlsx"), engine="openpyxl)
# df_resultados_elm["model"] = "ELM"

df_analise = pd.concat([
    df_resultados_svm,
    df_resultados_xgboost,
    df_resultados_mlp, 
#     df_resultados_elm,  
])
df_analise["transfer_learning_method"] = 'nilmnetA30d' # incluindo tipo de extrator utilizado (d30 = 30 dias de consumo)
df_analise.to_excel(os.path.join(caminho_dados_notebook, "df_analise.xlsx"))

print("* Análise por modelo:")
df_analise_modelo = df_analise.groupby(["model","base"]).agg({
    "acc": ["mean","std","max","min"],
    "f1": ["mean","std","max","min"],
    "auc": ["mean","std","max","min"]
}).reset_index().sort_values(('f1','mean'), ascending=False).set_index("model")
display(df_analise_modelo)
df_analise_modelo.to_excel(os.path.join(caminho_dados_notebook, "df_analise_modelo.xlsx"))

print()
print("* Análise por aparelho/modelo:")
df_analise_aparelho = df_analise.groupby(["appliance","model","base"]).agg({
    "acc": ["mean","std","max","min"],
    "f1": ["mean","std","max","min"],
    "auc": ["mean","std","max","min"]
})#.reset_index().sort_values(('f1','mean'), ascending=False).set_index(["aparelho","metodologia"])
display(df_analise_aparelho)
df_analise_aparelho.to_excel(os.path.join(caminho_dados_notebook, "df_analise_aparelho.xlsx"))

* Análise por modelo:


base       acc                                      f1            \
                     mean       std       max       min      mean       std   
model                                                                         
XGBOOST   teste  0.952416  0.032435  0.996908  0.891892  0.715973  0.207256   
XGBOOST  treino  0.951730  0.034133  1.000000  0.869565  0.707932  0.204972   
MLP       teste  0.952972  0.027486  0.996908  0.912162  0.629383  0.220078   
MLP      treino  0.945189  0.070467  1.000000  0.304348  0.595961  0.195828   
SVM       teste  0.945165  0.028555  0.996599  0.905405  0.583270  0.185560   
SVM      treino  0.949405  0.026031  1.000000  0.913043  0.576198  0.174336   

                                  auc                                
              max       min      mean       std       max       min  
model                                                                
XGBOOST  0.980255  0.471429  0.707410  0.195759  0.980255  0.478261  
XGBOOST  1.000000  0.465116  0.713315  0.204055  1.000000  0.454545  
MLP      0.979815  0.482517  0.632526  0.206556  0.969375  0.498724  
MLP      1.000000  0.292308  0.602731  0.178854  1.000000  0.489796  
SVM      0.977879  0.475177  0.587178  0.170254  0.969213  0.500000  
SVM      1.000000  0.477273  0.584115  0.162499  1.000000  0.500000


* Análise por aparelho/modelo:


acc                                \
                                      mean       std       max       min   
appliance         model   base                                             
dish_washer - 9   MLP     teste   0.932432  0.000000  0.932432  0.932432   
                          treino  0.947628  0.017752  0.956522  0.913043   
                  SVM     teste   0.932432  0.000000  0.932432  0.932432   
                          treino  0.947628  0.017752  0.956522  0.913043   
                  XGBOOST teste   0.908784  0.023889  0.925676  0.891892   
                          treino  0.930138  0.035725  1.000000  0.869565   
fridge - 7        MLP     teste   0.954878  0.001725  0.956098  0.953659   
                          treino  0.924120  0.010353  0.937500  0.906250   
                  SVM     teste   0.956098  0.000000  0.956098  0.956098   
                          treino  0.926463  0.007196  0.937500  0.921875   
                  XGBOOST teste   0.960976  0.006899  0.965854  0.956098   
                          treino  0.941307  0.020221  0.984375  0.921875   
microwave - 16    MLP     teste   0.939024  0.000000  0.939024  0.939024   
                          treino  0.944250  0.009397  0.960784  0.923077   
                  SVM     teste   0.939024  0.000000  0.939024  0.939024   
                          treino  0.945211  0.007996  0.960784  0.941176   
                  XGBOOST teste   0.935976  0.012935  0.945122  0.926829   
                          treino  0.933484  0.023212  0.980392  0.882353   
washer_dryer - 13 MLP     teste   0.992579  0.006122  0.996908  0.988250   
                          treino  0.992751  0.005324  1.000000  0.984095   
                  SVM     teste   0.989487  0.010058  0.996599  0.982375   
                          treino  0.990964  0.007274  1.000000  0.980119   
                  XGBOOST teste   0.993506  0.004810  0.996908  0.990105   
                          treino  0.992950  0.006361  1.000000  0.980159   
washer_dryer - 14 MLP     teste   0.945946  0.047777  0.979730  0.912162   
                          treino  0.917194  0.147152  1.000000  0.304348   
                  SVM     teste   0.908784  0.004778  0.912162  0.905405   
                          treino  0.936759  0.022016  0.956522  0.913043   
                  XGBOOST teste   0.962838  0.023889  0.979730  0.945946   
                          treino  0.960771  0.031243  1.000000  0.913043   

                                        f1                                \
                                      mean       std       max       min   
appliance         model   base                                             
dish_washer - 9   MLP     teste   0.482517  0.000000  0.482517  0.482517   
                          treino  0.486514  0.004743  0.488889  0.477273   
                  SVM     teste   0.482517  0.000000  0.482517  0.482517   
                          treino  0.486514  0.004743  0.488889  0.477273   
                  XGBOOST teste   0.476065  0.006557  0.480702  0.471429   
                          treino  0.523374  0.135895  1.000000  0.465116   
fridge - 7        MLP     teste   0.488459  0.000451  0.488778  0.488140   
                          treino  0.488588  0.038150  0.650273  0.475410   
                  SVM     teste   0.488778  0.000000  0.488778  0.488778   
                          treino  0.480907  0.001933  0.483871  0.479675   
                  XGBOOST teste   0.642703  0.217682  0.796627  0.488778   
                          treino  0.652334  0.181832  0.940243  0.479675   
microwave - 16    MLP     teste   0.484277  0.000000  0.484277  0.484277   
                          treino  0.485651  0.002480  0.490000  0.480000   
                  SVM     teste   0.484277  0.000000  0.484277  0.484277   
                          treino  0.485909  0.002101  0.490000  0.484848   
                  XGBOOST teste   0.634004  0.162055  0.748595  0.519414   
          

: 

# Conclusões

...

# Fim.

In [18]:
%load_ext watermark

In [19]:
%watermark -a "Diego Luiz Cavalca" -u -n -t -z -v -m -g

Author: Diego Luiz Cavalca

Last updated: Sun May 02 2021 13:36:14Hora oficial do Brasil

Python implementation: CPython
Python version       : 3.8.8
IPython version      : 7.21.0

Compiler    : MSC v.1928 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 158 Stepping 9, GenuineIntel
CPU cores   : 8
Architecture: 64bit

Git hash: a29eb3e98689f89f3597358428a2cab6bb3ab9b0

